In [4]:
import os.path
import re

from typing import List

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [9]:
# html = open('../data/pdfs/ROCNE SPRAVY  2021/ACROSS FUNDING správa 2021.xhtml').read()
html = open('../../../data/pdfs/ROCNE SPRAVY  2021/GEOCOMPLEX.xhtml').read()
# html = open('../data/pdfs/ROCNE SPRAVY  2021/Eurovea 2 financing sprava 2021.xhtml').read()
soup = BeautifulSoup(html)
[s.extract() for s in soup(['style', 'script', '[document]', 'head', 'title', 'img'])]
# all_content = soup.find('div', attrs={'id': "page-container"})
visible_text = soup.getText()
# print(visible_text)
print(len(soup.findAll('p')))

7297


In [10]:
from bs4 import NavigableString, Tag


def most_frequent(arr):
    dict = {}
    count, itm = 0, ''
    for item in reversed(arr):
        dict[item] = dict.get(item, 0) + 1
        if dict[item] >= count :
            count, itm = dict[item], item
    return(itm)

def get_class_meta_from_div(found_classes: List, class_meta_name: str):
    # print(tag['class'])
    # print(class_meta_name)
    ret_list = [class_meta.split(class_meta_name)[1] for class_meta in found_classes if class_meta_name in class_meta]
    return ret_list if len(ret_list)>0 else [-1]

texts = {}
class_meta_names = ["P", "T"]
text_count = -1
br_cnt = 0

for j, paragraph in enumerate(soup.findAll('p')):

    paragraph_text = paragraph.getText().strip()

    if not paragraph_text or all(not x.isalpha() for x in paragraph_text):
        continue

    classes = paragraph['class']
    meta_info = {}

    if paragraph.find('span'):
        other_classes = []
        for j, span in enumerate(paragraph.findAll('span')):

            span_text = span.getText().strip()

            if not span.has_attr('class') or \
                    not span_text or \
                    all(not x.isalpha() for x in span_text):
                continue
            classes += span['class']

            meta_info = {}
            other_classes = [cls for cls in classes if not cls.startswith(tuple(class_meta_names))]

            for class_meta_name in class_meta_names:
                meta_info[class_meta_name] = get_class_meta_from_div(classes, class_meta_name)

    else:
        other_classes = [cls for cls in classes if not cls.startswith(tuple(class_meta_names))]
        for class_meta_name in class_meta_names:
            meta_info[class_meta_name] = get_class_meta_from_div(classes, class_meta_name)

    text_blocks = []
    if paragraph.find('br'):
        for br in paragraph.findAll('br'):
            next_s = br.nextSibling
            if not (next_s and isinstance(next_s,NavigableString)):
                continue
            next2_s = next_s.nextSibling
            if next2_s and isinstance(next2_s,Tag) and next2_s.name == 'br':
                text = str(next_s).strip()
                if text:
                    text_blocks.append(next_s)
    else:
        text_blocks.append(paragraph_text)

    for extracted_text in text_blocks:
        text_count+=1
        texts[text_count]={}
        texts[text_count]["text"]=extracted_text
        texts[text_count]['page'] = 1
        texts[text_count]["other_classes"] = list(set(other_classes))
        for class_meta_name in class_meta_names:
            if class_meta_name in meta_info.keys():
                texts[text_count][class_meta_name] =  most_frequent(meta_info[class_meta_name])
            else:
                texts[text_count][class_meta_name] = -1

doc_df = pd.DataFrame.from_dict(texts, orient='index')

display(doc_df)

,text,page,other_classes,P,T
0,"GEOCOMPLEX, a.s.",1,[],516,-1
1,v zmysle ustanovenie § 34 ods. 2 pís. c) zákon...,1,[],1,-1
2,"Geocomplex, a. s., Twin City Tower, Mlynské Ni...",1,[],97,12
3,"Obchodný register : Okresný súd Bratislava I, ...",1,[],20,-1
4,"Tel., fax : 02/4552 0478, e-mail : geocomp...",1,[],20,-1
...,...,...,...,...,...
2154,a)použitá metóda vykazovania peňažných tokov z...,1,[ListLabel_20_368],575,-1
2155,1 § 20 ods. 7 písm. f) zákona o účtovníctve,1,[],-1,-1
2156,2 § 20 ods. 7 písm. d) zákona o účtovníctve,1,[],-1,-1
2157,3 § 20 ods. 6 písm. b) a f) a § 20 ods. 7 pís...,1,[],-1,-1


In [11]:
for text in doc_df['text']:
    print(text)
    print("---------------")

GEOCOMPLEX, a.s.
---------------
v zmysle ustanovenie § 34 ods. 2 pís. c) zákona 429/2002 Z.z.
---------------
Geocomplex, a. s., Twin City Tower, Mlynské Nivy 10, 821 09 Bratislava, IČO : 31 322 638, IČ DPH : SK2020389910
---------------
Obchodný register : Okresný súd Bratislava I, oddiel Sa, vložka číslo 413/B
---------------
Tel., fax :  02/4552 0478,    e-mail : geocomplex@geocomplex.sk,    www.geocomplex.sk
---------------
Bankové spojenie: Poštová banka, a. s.,  č. účtu: 20550376/6500
---------------
Spoločnosť pre komplexné riešenie geofyzikálnych, geologických a ekologických problémov
---------------
ku ročnej finančnej správe za rok 2021 v zmysle ustanovenie § 34 ods. 2 pís. c) zákona 429/2002 Z.z.
---------------
Predstavenstvo spoločnosti GEOCOMPLEX, a. s., v zložení: predseda predstavenstva - JUDr. Juraj Vrábel, člen predstavenstva – RNDr. Viktoria Szalaiová a člen predstavenstva - Martina Hudáková, vyhlasuje, že podľa ich najlepších znalostí poskytuje účtovná závierka vyp

In [13]:
def get_number_of_words(row):
    return len(row['text'].split())

def get_number_of_chars(row):
    return len(row['text'])

def get_number_of_alphachars(row):
    return sum(c.isalpha() for c in row['text'])

def store_text_info_only(dataframe_elements, out_firm_dir):
    # mask = (dataframe_elements.label == 'text')
    # dataframe_texts = dataframe_elements.copy()
    dataframe_elements['words'] = dataframe_elements.apply(get_number_of_words, axis=1)
    dataframe_elements['chars'] = dataframe_elements.apply(get_number_of_chars, axis=1)
    dataframe_elements['alpha'] = dataframe_elements.apply(get_number_of_alphachars, axis=1)
    dataframe_elements.to_csv(os.path.join(out_firm_dir, "all_texts_info.csv"))

    return dataframe_elements, os.path.join(out_firm_dir, "all_texts_info.csv")

dataframe_elements, dataframe_texts_csv_path = store_text_info_only(doc_df, "../../../data/out/")

In [16]:
def get_param_stats(dataframe_elements, param_name):
    dataframe_elements['index1'] = dataframe_elements.index
    # for param_val in dataframe_elements[param_name]:
        # print(param_val)
    # print([dataframe_elements[dataframe_elements[param_name] in param_val] for param_val in dataframe_elements[param_name] if param_val is not None ])
    param_counts = [ dataframe_elements[param_val == dataframe_elements[param_name]]["index1"].count() for param_val in
              dataframe_elements[param_name]]
    param_mean_word_count = [ dataframe_elements[param_val == dataframe_elements[param_name]]["words"].mean() for param_val in
              dataframe_elements[param_name]]
    # print(param_mean_word_count)
    sum_of_all_texts = dataframe_elements["index1"].count()
    dataframe_elements[f'{param_name}_frequency'] = param_counts / sum_of_all_texts
    dataframe_elements[f'{param_name}_mean_word_count'] = param_mean_word_count
    return dataframe_elements

dataframe_elements = get_param_stats(dataframe_elements, 'P')
dataframe_elements = get_param_stats(dataframe_elements, 'T')

display(dataframe_elements)
doc_df.to_csv("../data/out/test.csv")

# def store_base_file_stat(dataframe_elements, out_firm_dir):
#     dataframe_elements['index1'] = dataframe_elements.index
#     fs_counts = [dataframe_elements[dataframe_elements['fs'] == size]["index1"].count() for size in
#               dataframe_elements['fs']]
#     sum_of_all_texts = dataframe_elements["index1"].count()
#     dataframe_elements['size_frequency'] = counts / sum_of_all_texts
#     dataframe_elements.to_csv(os.path.join(out_firm_dir, "base_file_stat.csv"), index=False)
#
#     return dataframe_elements, os.path.join(out_firm_dir, "base_file_stat.csv")

516
1
97
20
20
21
511
530
222
-1
-1
-1
-1
-1
-1
-1
93
-1
93
226
223
223
223
225
229
229
229
232
232
229
234
229
234
229
230
234
230
230
234
229
234
229
229
229
229
229
229
229
234
229
234
229
229
229
234
33
229
198
229
33
229
225
33
234
229
229
234
236
229
234
229
234
229
234
229
234
229
234
229
234
238
229
229
234
84
229
234
199
229
234
239
229
234
229
234
229
234
33
234
224
229
229
229
229
33
234
229
229
234
229
234
229
234
229
234
229
234
225
225
225
241
229
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
229
234
229
229
234
33
234
229
234
229
198
229
234
241
229
229
198
229
234
241
234
229
229
33
229
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
33
229
234
229
234
234
229
234
234
229
234
229
229
234
198
236
231
231
231
231
231
231
231
235
235
235
235
235
250
250
250
250
250
250
250
231
231
231
235
235
33
234
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
229
239
234
229
198
229
234
229
234
229
234
229
234
229
198
229
234
224
216
234
223
436
223
228
234

TypeError: 'in <string>' requires string as left operand, not Series

In [7]:
def store_unique_formatting_stats(dataframe_elements, out_firm_dir):
    dataframe_unique_formatting = dataframe_elements.groupby([ 'P', 'T', 'P_frequency', 'T_frequency',  'P_mean_word_count', 'T_mean_word_count']).agg({
        'text': list,
        "page": list,
        'index1': [list, 'count']})
    dataframe_unique_formatting.columns = ['_'.join(col).strip() for col in
                                           dataframe_unique_formatting.columns.values]
    dataframe_unique_formatting.reset_index(inplace=True)
    dataframe_unique_formatting.to_csv(os.path.join(out_firm_dir, "unique_formatting.csv"), index=True)

    return dataframe_unique_formatting, os.path.join(out_firm_dir, "unique_formatting.csv")

In [8]:
dataframe_unique_formatting, dataframe_unique_formatting_csv_path = store_unique_formatting_stats(dataframe_elements, "../data/out/")

In [9]:
def give_label(row):
    if row['P_frequency'] == 0 or row['T_frequency'] == 0:
        return "other"
    elif row['P_mean_word_count'] > 7 or row['T_mean_word_count'] > 7:
        return "text"
    elif row['P_frequency'] > 0.01 or row['T_frequency'] > 0.01:
        return "header"
    else:
        return "other"
    # zvysok by mal byt trash -- nadpisy v tabulkach, chyby pri analyze pdf ...
    # return None

dataframe_unique_formatting['label'] = dataframe_unique_formatting.apply(give_label, axis=1)
dataframe_unique_formatting.to_csv(dataframe_unique_formatting_csv_path, index=True)
doc_df['label'] = doc_df.apply(give_label, axis=1)
doc_df.to_csv("../data/out/labelled.csv")

display(doc_df)

,text,page,other_classes,P,T,words,chars,alpha,index1,P_frequency,P_mean_word_count,T_frequency,T_mean_word_count,label
0,"Eurovea 2 financing, s.r.o.",1,[],54,1,4,27,19,0,0.000693,4.000000,0.000693,4.000000,other
1,Správa nezávislého audítora o overení účtovnej...,1,[],175,3,8,57,50,1,0.000693,8.000000,0.004161,6.166667,text
2,Správa k ďalším požiadavkám zákona a iných prá...,1,[],176,3,10,63,54,2,0.000693,10.000000,0.004161,6.166667,text
3,Výročná správa 2021,1,[],179,3,3,19,13,3,0.000693,3.000000,0.004161,6.166667,other
4,Obsah,1,[],73,2,1,5,5,4,0.000693,1.000000,0.000693,1.000000,other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1437,Účtovná závierka Spoločnosti bola overená audí...,1,[],218,51,6,52,46,1437,0.002080,30.666667,0.002080,30.666667,text
1438,V Bratislave dňa 22.4.2022,1,[],178,87,4,26,14,1438,0.002774,2.750000,0.029126,9.571429,text
1439,Marek Lenčéš,1,[ra],178,88,2,12,11,1439,0.002774,2.750000,0.000693,2.000000,other
1440,konateľ,1,[ra],178,87,1,7,7,1440,0.002774,2.750000,0.029126,9.571429,text


In [10]:
last_header = ''
headers = []
for text, label in zip(doc_df.text, doc_df.label):
    if label == 'header':
        last_header = text
    headers.append(last_header)
doc_df['belonging_header'] = headers
doc_df.to_csv("../data/out/assingned_headers.csv")

In [11]:
def filter_texts(dataframe_texts, out_firm_dir):
    mask2 = (dataframe_texts.alpha > 40) & (dataframe_texts.words > 3)
    dataframe_texts_clean = dataframe_texts[mask2].copy()
    dataframe_texts_clean['d'] = 0
    dataframe_texts_clean['h'] = 0
    dataframe_texts_clean.to_csv(os.path.join(out_firm_dir, "filtered_texts_info.csv"))
    return dataframe_texts_clean, os.path.join(out_firm_dir, "filtered_texts_info.csv")
dataframe_texts_clean, dataframe_texts_clean_csv_path = filter_texts(doc_df, "../data/out/")